In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
print(tf.__version__)

tf.keras.backend.clear_session()  # For easy reset of notebook state.

import pandas as pd
import numpy as np
import xarray as xr

In [ ]:
import os

df = {}
file_path = os.path.join(os.getcwd(), "forex-data")
for f in os.listdir(file_path):
    if f.startswith('USD'):
        df[f] = pd.read_csv(os.path.join(file_path, f))
        df[f]['openTime'] = pd.to_datetime(df[f].Date, unit='ns')
        df[f]['openTime'] = pd.DatetimeIndex(df[f]['openTime'] ).round('S')
        df[f].index = df[f]['openTime']
        df[f].drop(columns=['openTime', 'Date'])
        df[f] = xr.DataArray(df[f], dims=['openTime', 'feature']) 
        
chart = xr.Dataset(df).to_array(dim='asset')
chart = chart.transpose("feature", "asset", "openTime")

print(chart.shape)
print(chart)

In [ ]:
from pgportfolio.learn.nnagent import NNAgent
from pgportfolio.tools.configprocess import load_config

datetime = pd.date_range(start="01/01/2015", end="30/12/2016", freq="10min")

In [ ]:
from pgportfolio.learn.nnagent import NNAgent
from pgportfolio.tools.configprocess import load_config
from datetime import datetime, timedelta


def assets2symbols(assets):
    return [asset+"/USD" for asset in assets]

def backtest(start, end, agent_index = 1,location = "train_package/"):

    config = load_config(agent_index)
    assets = ["CHF", "JPY", "CAD", "DKK"]
    agent = NNAgent(config, location+str(agent_index)+"/netfile")

    commission_rate = config["trading"]["trading_consumption"]

    window_size = config["input"]["window_size"]
    global_steps = len(global_data["openTime"]) - window_size
    last_w = np.zeros(len(assets)+1)
    last_w[0] = 1
    portfolio_changes_history = {}
    portfolio_weights_history = {}
    print(global_data)
    for i in range(global_steps):
        input_window = global_data[:,:,i:i+window_size]
        y = global_data[0, :, i+1]/global_data[0, :, i]
        datetime = input_window["openTime"][-1].values
        weights = agent.decide_by_history(input_window.values, last_w)
        transaction_cost = np.sum(np.abs(weights-last_w))*commission_rate
        last_w = weights
        portfolio_change = np.dot(y, weights[1:])*(1-transaction_cost)
        portfolio_changes_history[datetime] = portfolio_change
        portfolio_weights_history[datetime] = weights
    return {"portfolio_changes_history": pd.Series(portfolio_changes_history),
            "portfolio_weights_history": pd.DataFrame(portfolio_weights_history),
            "global_data": global_data}


results = backtest("2015-01-01 00:10:00", "2016-01-01 01:00:00")
print(results)